In [4]:
import numpy as np
import math
import pathlib
import pandas as pd
import os

from tensorflow import keras

import plotly.express as px
import plotly.graph_objects as go

import datetime

# models

In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error



In [6]:
# Continuar desenvolvimento:
# https://plotly.com/python/line-charts/#line-plots-with-column-encoding-color

def show_copare_prediction():
	fig = px.line(df, x='index', y='predictions', title='Ibova predito')
	fig.add_scatter(x=df['index'], y=df['real_values'], mode='lines')
	fig.show()

def show_distribution_error():
	fig = px.histogram(df,x='error', )
	fig.show()

# https://www.askpython.com/python/examples/mape-mean-absolute-percentage-error
# def MAPE(Y_actual,Y_Predicted):
# 	# ((Y_actual - Y_Predicted)
# 	# Soma os erros
# 	# / n 
# 	# # Calculando mae ^^
# 	# somatoria absolute error /n

# 	Y_actual, Y_Predicted = np.array(Y_actual), np.array(Y_Predicted)
# 	return np.mean(np.abs((Y_actual - Y_Predicted) / Y_actual)) * 100

# 	np.subtract(Y_actual,Y_Predicted)

	# teste = np.abs((Y_Predicted - Y_actual)/(Y_actual))
	# # teste = np.abs((Y_actual - Y_Predicted)/(Y_actual + 1))*100
	# # # teste = teste[1:]
	# somatorio = np.sum(teste)
	# MAPE = somatorio/int(len(teste))
	# # MAPE = np.abs(teste/475)
	# # print (f"The MAPE: {MAPE}")
	# return MAPE

# erro ²²
# MSE  = Soma dos erros²/n

# https://www.askpython.com/python/examples/rmse-root-mean-square-error
# def RMSE(Y_actual,Y_Predicted):
# 	MSE = np.square(np.subtract(Y_actual,Y_Predicted)).mean()
# 	# print (f"The MSE: {MSE}")
# 	RMSE = math.sqrt(MSE)
# 	# print (f"The RMSE: +- {RMSE}")
# 	return RMSE


def MAPE(Y_actual,Y_Predicted):
	# mape = mean_absolute_percentage_error(Y_actual, Y_Predicted)

	teste = np.abs((Y_Predicted - Y_actual)/(Y_actual))*100
	# teste = teste[1:]
	MAPE = np.mean(teste)
	# print (f"The MAPE: {MAPE}")
	# mape = mean_absolute_percentage_error(Y_Predicted, Y_actual)
	return MAPE
	# return mape 


def RMSE(Y_actual,Y_Predicted):

	# rmse = mean_squared_error(Y_actual, Y_Predicted, squared = False)
	# return rmse

	MSE = np.square(np.subtract(Y_actual,Y_Predicted)).mean()
	# print (f"The MSE: {MSE}")
	RMSE = math.sqrt(MSE)
	print (f"The RMSE: +- {RMSE}")
	return RMSE
	


# https://www.askpython.com/python/examples/rmse-root-mean-square-error
# def RMSE(Y_actual,Y_Predicted):
# 	MSE = np.square(np.subtract(Y_actual,Y_Predicted)).mean()
# 	# print (f"The MSE: {MSE}")
# 	RMSE = math.sqrt(MSE)
# 	# print (f"The RMSE: +- {RMSE}")
# 	return RMSE


# def MAPE(actual, predicted) -> float:
  
    # Convert actual and predicted
    # to numpy array data type if not already
    # if not all([isinstance(actual, np.ndarray),isinstance(predicted, np.ndarray)]):
    #     actual, predicted = np.array(actual), 
    #     np.array(predicted)
  
    # Calculate the MAPE value and return
    # return round(np.mean(np.abs((
    #   actual - predicted) / actual)) * 100, 2)



In [7]:
# Reading DF teste
PATH_TESTE = '../Data/3_Gold/2_Teste_all_stocks.csv'
# PATH_TESTE = '../Data/3_Gold/Teste_all_stocks.csv'
DF_TESTE = pd.read_csv(PATH_TESTE, sep=",")
test_dataset = DF_TESTE

test_dataset = test_dataset.drop(['oil_5','usd_5','petr_5','vale_5',],axis=1)
test_labels = test_dataset.pop('ibova_5')

In [8]:
TYPE_MODEL = 'MLP'
# MODELS_PATH = f'../Experiments/Models/{TYPE_MODEL}/'
MODELS_PATH = f'../Experiments/Models/{TYPE_MODEL}/'
SAVE_TIME = datetime.datetime.strftime(datetime.datetime.now(), "%y_%m_%d_%Hh%Mm%S")

models = os.listdir(MODELS_PATH)

# Carregando os modelos 
df_results = pd.DataFrame(columns=['path_model','MAPE','RMSE','model_type'])

for id, model_path in enumerate(models):
	print(model_path)

	model = keras.models.load_model(f'../Experiments/Models/{TYPE_MODEL}/{model_path}')
	# Model\Models\MLP
	try:
		test_predictions = model.predict(test_dataset).flatten()
		# print(test_predictions)
	except Exception as e:
		print(e)

	# Definindo o Dfs
	df_predic = pd.DataFrame(test_predictions, columns=['predictions'])
	df_predic = df_predic.reset_index()

	df_label = pd.DataFrame(test_labels.values, columns=['real_values'])
	df_label = df_label.reset_index()

	# Validação da quantidade de resultados
	# predict_len = len(test_predictions)
	# print(f'Predict len: {predict_len}')

	error = test_predictions - test_labels
	df_error = pd.DataFrame(error.values, columns=['error'])
	df_error = df_error.reset_index()

	# Merging Df
	df = df_predic.merge(df_label)
	df = df.merge(df_error)

	# Teste 
	# df.loc[id,'model_type'] = TYPE_MODEL
	# df.to_csv(f'Predictions_{TYPE_MODEL}_{SAVE_TIME}.csv',header=True,)
	# exit()

	# Get metrics and saving results
	mape = MAPE(df['real_values'], df['predictions'])
	rmse = RMSE(df['real_values'], df['predictions'])
	print(mape)
	print(rmse)

	df_results.loc[id,'path_model'] = model_path
	df_results.loc[id,'MAPE'] = mape
	df_results.loc[id,'RMSE'] = rmse
	df_results.loc[id,'model_type'] = TYPE_MODEL
	# df_results = df_results.reset_index()


model_MLP_22_12_01_21h45m20.h5
18/18 [==============================] - 1s 2ms/step
The RMSE: +- 2.2463554978651668
274.7065139696763
2.2463554978651668
model_MLP_22_12_01_21h47m59.h5
18/18 [==============================] - 0s 2ms/step
The RMSE: +- 2.1977128296466835
225.4221649053429
2.1977128296466835
model_MLP_22_12_01_21h50m34.h5
18/18 [==============================] - 0s 2ms/step
The RMSE: +- 2.128467181151841
151.51569888780253
2.128467181151841
model_MLP_22_12_01_21h53m10.h5
18/18 [==============================] - 0s 2ms/step
The RMSE: +- 2.086261889825781
152.49140941543808
2.086261889825781
model_MLP_22_12_01_21h55m20.h5
18/18 [==============================] - 0s 2ms/step
The RMSE: +- 2.218577580938398
224.45761344537854
2.218577580938398
model_MLP_22_12_01_21h57m27.h5
18/18 [==============================] - 0s 2ms/step
The RMSE: +- 2.079271059220835
140.259679616418
2.079271059220835
model_MLP_22_12_01_21h59m35.h5
18/18 [==============================] - 0s 2ms/step
The 

In [9]:
df_results.to_csv(f'Indicators_{TYPE_MODEL}_{SAVE_TIME}.csv',header=True,)
# df_results = None

In [10]:


# https://www.askpython.com/python/examples/mape-mean-absolute-percentage-error



#

In [11]:
# Usar só mse e mape na validacao 

In [12]:
# TODO: ver de plotar o RMSE pra cima e pra baixo 
show_copare_prediction()

Tirar o titulo do grafico 

deixar tudo em portugues 

y = ROC
Legenda a linha azul e vermelha - real vs predito

Chamar index de days 

In [13]:
show_distribution_error()